In [ ]:
import json
import keras
import keras_nlp
import pickle
import tensorflow as tf

In [ ]:
# File names
CUSTOM_MODEL = "/content/custom_model.keras"
VOCAB_FILE = "/content/vocab.pickle"
LORA_MODEL = "/content/transfer_learning.lora.h5"

SEQ_LEN = 512  # Length of training sequences, in tokens. AKA the context size
GPT2_PRESET = "gpt2_base_en" # name of base model

# Special tokens
START_OF_RECIPE = "<|recipe_start|>"
END_OF_RECIPE = "<|recipe_end|>"
PAD = "<|pad|>"
OOV = "<|oov|>"
SPECIAL_TOKENS = [PAD, START_OF_RECIPE, END_OF_RECIPE, OOV]

# Generating recipes with the custom model

In [ ]:
custom_model = keras.models.load_model(CUSTOM_MODEL)

with open(VOCAB_FILE, "rb") as f:
    vocab = pickle.load(f)

custom_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    special_tokens_in_strings=True,
    special_tokens=SPECIAL_TOKENS,
    oov_token=OOV,
)

packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=custom_tokenizer.token_to_id(START_OF_RECIPE),
    end_value=custom_tokenizer.token_to_id(END_OF_RECIPE),
    pad_value=custom_tokenizer.token_to_id(PAD),
)
custom_model.summary()

In [ ]:
class CustomTextGenerator():
    def __init__(self, model, p):
        self.model = model
        self.sampler = keras_nlp.samplers.TopPSampler(p=p, k=1024)

    def _tokenize_str(self, str):
        return packer(custom_tokenizer([str]))

    def _next(self, prompt, cache, index):
        logits = self.model(prompt)[:, index-1, :]
        hidden_states = None,
        return logits, hidden_states, cache
    
    def _normalize_output(self, txt):
        txt = txt.split(END_OF_RECIPE)[0].split('}')[0]  + '}'
        txt = txt.replace(START_OF_RECIPE, "").replace(PAD, "")
        txt = txt.replace(OOV, "").replace(' " ', '"')
        try:
            txt = json.dumps(json.loads(txt), indent=4)
        except Exception as _:
            txt = "Unable to parse as a JSON object\n" + txt
        return txt

    def generate(self, seed_text, logs=None):
        seed_tokens = self._tokenize_str(seed_text)
        seed_length = tf.reduce_sum(tf.cast(~tf.equal(seed_tokens, 0), tf.int8)).numpy()
        output_tokens = self.sampler(
            next=self._next,
            prompt=seed_tokens,
            index=seed_length,
        )
        txt = custom_tokenizer.detokenize(output_tokens).numpy()
        txt = txt[0].decode("utf-8")
        txt = self._normalize_output(txt)
        return txt
    
    def generate_recipe(self, ingredients):
        seed_text = '{"ner": ['
        for ingredient in ingredients[:-1]:
            seed_text += f'"{ingredient}", ' 
        seed_text += f'"{ingredients[-1]}","' if ingredients else ''
        
        return self.generate(seed_text)
    
custom_generator = CustomTextGenerator(custom_model, 0.9)

In [ ]:
seed_ingredients = ["chicken", "tomato", "pasta"]
recipe = custom_generator.generate_recipe(seed_ingredients)
print(recipe)

# Generating recipes with fine-tuned GPT-2

In [ ]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    GPT2_PRESET,
    sequence_length=SEQ_LEN,
)
lora_model = keras_nlp.models.GPT2CausalLM.from_preset(
    GPT2_PRESET,
    preprocessor=preprocessor,
)
lora_model.backbone.load_lora_weights(LORA_MODEL)
lora_model.summary()

In [ ]:
class LoRATextGenerator():
  def __init__(self, model):
    self.model = model

  def _normalize_output(self, txt):
    try:
        txt = json.dumps(json.loads(txt), indent=4)
    except Exception as _:
        txt = "Unable to parse as a JSON object\n" + txt
    return txt

  def generate(self, seed_text):
    raw_output = self.model.generate(seed_text, max_length=SEQ_LEN)
    output = self._normalize_output(raw_output)
    return output

  def generate_recipe(self, ingredients):
    seed_text = '{"ner": ['
    for ingredient in ingredients[:-1]:
        seed_text += f'"{ingredient}", ' 
    seed_text += f'"{ingredients[-1]}","' if ingredients else ''

    return self.generate(seed_text)

lora_generator = LoRATextGenerator(lora_model)

In [ ]:
lora_reicpe = lora_generator.generate_recipe(seed_ingredients)
print(lora_reicpe)

# Speed comparison

In [ ]:
import time

custom_start_time = time.time()
custom_random_recipe = custom_generator.generate_recipe([])
custom_end_time = time.time()
lora_start_time = time.time()
lora_random_recipe = lora_generator.generate_recipe([])
lora_end_time = time.time()

custom_time = custom_end_time - custom_start_time
lora_time = lora_end_time - lora_start_time

print(f"Custom model took {custom_time:.2f} seconds to generate a recipe")
print(f"LoRA model took   {lora_time:.2f} seconds to generate a recipe")

print(f"\nCustom model's output:\n{custom_random_recipe}")
print(f"\nLoRA model's output:\n{lora_random_recipe}")